**6.1. ГЕНЕРАТОРЫ И YIELD**

In [2]:
def countdown(n):
    print('Counting from down', n)
    while n > 0:
        yield n
        n -= 1
        
# Пример использования
for x in countdown(10):
    print('T-minus', x)

Counting from down 10
T-minus 10
T-minus 9
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


In [5]:
c = countdown(10)
print(c)
# Один из способов вывода - next
print(next(c))

# next - сокращение вызова метода __next__()

print(c.__next__())
print(c.__next__())

a = sum(countdown(10))
print(a)

<generator object countdown at 0x7f9f7c5093c0>
Counting from down 10
10
9
8
Counting from down 10
55


In [54]:
# Следующая функция использует как yield, так и return

def func():
    yield 37
    return 42

f = func()

try:
    print(next(f))
    print(next(f))
except StopIteration as e:
    print(e.value)
            
        


37
42


In [20]:
# Рассмотрим пример с вызовом break
for n in countdown(10):
    if n == 2:
        break
        
# Цикл for прерывается вызовом break и связанный с ним генератор никогда не
# отрабатывает до завершения

# Используем try-finally или менеджер контекста

def countdown(n):
    print('Counting down from', n)
    try:
        while n > 0:
            yield n
            n -= 1
    finally:
        print('Only made it to', n)
        
for i in countdown(5):
    print(i)
    
    
# Менеджер контекста также завершает любой код

def func(filename):
    with opne(filename) as file:
        ...
        yield data
        ...
        # Здесь файл будет закрыт, даже если генератор был прерван

Counting down from 10
Only made it to 2
Counting down from 5
5
4
3
2
1
Only made it to 0


**6.2. ПЕРЕЗАПУСКАЕМЫЕ ГЕНЕРАТОРЫ**

In [56]:
c = countdown(3)
for n in c:
    print('T-minus', n)
for n in c:
    print('T-minus', n)

Counting down from 3
T-minus 3
T-minus 2
T-minus 1
Only made it to 0


>Чтобы создать объект, допускающий повторные итерации, определите его
как класс и сделайте метод __iter__() генератором:

In [57]:
class Countdown:
    def __init__(self, start):
        self.start = start
    def __iter__(self):
        self.start = start
        while n > 0:
            yield n
            n -= 1

**6.3. ДЕЛЕГИРОВАНИЕ**

In [58]:
def countup(stop):
    n = 1
    while n <= stop:
        yield n
        n += 1
        
def countdown(start):
    n = start
    while n > 0:
        yield n
        n -= 1

def up_and_down(n):
    yield from countup(n)
    yield from countdown(n)

In [59]:
def flatten(items):
    for i in items:
        if isinstance(i, list):
            yield from flatten(i)
        else:
            yield i
            
a =  [1, 2, [3, [4, 5], 6, 7], 8]
for x in flatten(a):
    print(x, end=' ')

1 2 3 4 5 6 7 8 

**6.4. ПРАКТИЧЕСКОЕ ИСПОЛЬЗОВАНИЕ
ГЕНЕРАТОРОВ**

>Генераторы могут стать полезным инструментом реструктуризации кода,
состоящего из глубоко вложенных циклов for и условных конструкций.
Рассмотрим сценарий, который ищет в каталоге с файлами Python все комментарии со словом spam:

In [ ]:
import pathlib
import re

for path in pathlib.Path('.').rglob('*.py'):
    if path.exists():
        with path.open('rt', encoding='latin-1') as file:
            for line in file:
                m = re.match('.*(#.*)$', line)
                if m:
                    comment = m.group(1)
                    if 'spam' in comment:
                        print(comment)
            

>Обратите внимание на глубину вложенности встроенных конструкций. Даже
от одного взгляда на этот код становится не по себе. А теперь взгляните на
следующую версию с использованием генераторов:

In [61]:
import pathlib
import re

def get_paths(topdir, pattern):
    for path in pathlib.Path(topdir).rglob(pattern):
        if path.exists():
            yield path
            
def get_files(paths):
    for path in paths:
        with path.open('rt', encoding='latin-1') as file:
            yield file
            
def get_lines(files):
    for file in files:
        yield from file
        
def get_comments(lines):
    for line in lines:
        m = re.match('.*(#.*)$', line)
        if m:
            yield m.group(1)
            
def print_matching(lines, substring):
    for line in lines:
        if substring in line:
            print(substring)
            
paths = get_paths('.', '*.py')
files = get_files(paths)
lines = get_lines(files)
comments = get_comments(lines)
print_matching(comments, 'spam')

In [ ]:
def flatten(items):
    for i in items:
        if isinstance(i, list):
            yield from flatten(i)
        else:
            yield i
            
def flatten(items):
    stack = [iter(items)]
    while stack:
        try:
            item = next(stack[-1])
            if isinstance(item, list):
                stack.append(iter(item))
            else:
                yield item
        except StopIteration:
            stack.pop()

**6.5. РАСШИРЕННЫЕ ГЕНЕРАТОРЫ
И ВЫРАЖЕНИЯ YIELD**

In [63]:
def receiver():
    print('Ready to receive')
    while True:
        n = yield
        print('Got', n)
        
r = receiver()
print(r.send(None))
print(r.send(1))
print(r.send('Hello'))
r.close()
r.send(4)

Ready to receive
None
Got 1
None
Got Hello
None


StopIteration: 

In [66]:
r = receiver()
r.throw(RuntimeError, 'Dead')

RuntimeError: Dead

**6.6. ПРИМЕНЕНИЕ РАСШИРЕННЫХ
ГЕНЕРАТОРОВ**

In [68]:
from contextlib import contextmanager
@contextmanager
def manager():
    print('Entering')
    try:
        yield 'somevalue'
    except Exception as e:
        print('An error occured', e)
    finally:
        print('Leaving')
        
with manager() as val:
    print(int(val))

Entering
An error occured invalid literal for int() with base 10: 'somevalue'
Leaving


In [71]:
# Для реализации этой функциональности используется класс-обертка.
# Следующая упрощенная реализация показывает основную идею:

class Manager:
    def __init__(self, gen):
        self.gen = gen
        
    def __enter__(self):
        return self.gen.send(None)
    
    def __exit__(self, ty, val, tb):
        try:
            if ty:
                try:
                    self.gen.throw(ty, val, tb)
                except ty:
                    return False
            else:
                self.gen.send(None)
        except StopIteration:
            return True


SyntaxError: invalid syntax (2118738213.py, line 24)

In [75]:
def line_receiver():
    data = bytearray()
    line = None
    linecount = 0
    while True:
        part = yield line
        linecount += part.count(b'\n')
        data.extend(part)
        print(data)
        if linecount > 0:
            index = data.index(b'\n')
            line = bytes(data[:index+1])
            data = data[index+1:]
            linecount -= 1
        else:
            line = None
            
r = line_receiver()
r.send(None)
r.send(b'hello')
r.send(b'world\nit ')

bytearray(b'hello')
bytearray(b'helloworld\nit ')


b'helloworld\n'

In [ ]:
# Такой код может быть записан в виде класса:

class LineReceiver:
    def __init__(self):
        self.data = bytearray()
        self.linecount = 0
        
    def send(self, part):
        self.linecount += part.count(b'\n')
        self.data.extend(part)
        if linecount > 0:
            index = self.data.index(b'\n')
            line = bytes(self.data[:index+1])
            self.data = self.data[index+1:]
            return line
        else:
            return None

**6.7. ГЕНЕРАТОРЫ И ИХ СВЯЗЬ С AWAIT**

In [102]:
# await использует скрытое взаимодействие с генератором. Далее показан
# базовый протокол, используемый await:

class Awaitable:
    def __await__(self):
        print('About to await')
        yield 'somevalue'
        print('Resuming')
        
# Функция, совместимая с await. Возвращает awaitable

def function():
    return Awaitable()

async def main():
    await function()
    
r = await main()
print(r)
# r.send('Hello')
    
# А вот как можно опробовать этот код с использованием asyncio:
import asyncio 
# asyncio.run(main())

About to await


RuntimeError: Task got bad yield: 'somevalue'

In [105]:
async def greeting(name):
    print(f'Hello {name}')
    
import asyncio
# asyncio.run(greeting('Guido'))
await greeting('Guido')

Hello Guido


In [107]:
async def make_greeting(name):
    return f'Hello {name}'

async def main():
    for name in ['Paula', 'Thomas', 'Lewis']:
        a = await make_greeting(name)
        print(a)
        
await main()

Hello Paula
Hello Thomas
Hello Lewis


In [108]:
async def twice(x):
    return 2 * x

def main():
    print(twice(2))
    print(await twice(2))

SyntaxError: 'await' outside async function (2242395077.py, line 6)

In [128]:
def fun():
    line = None
    while True:
        x = yield line
        if isinstance(x, int):
            x = 'OK'
        elif isinstance(x, str):
            x = 'OKOK'
r = fun()
r.send(None)
r.send('line')
print(r.send(3))

None
